In [ ]:
import xarray as xr
import cftime

min_lat, max_lat = -33, -24
min_lon, max_lon = -53, -44


def get_datetime(ds):
    date_time = cftime.num2date(
        times=ds["time"],
        units=ds["time"].units,
        calendar=ds["time"].calendar,
    )
    return date_time.strftime("%Y-%m-%d")


def slice_daily_roi(fname):
    ds = xr.open_dataset(fname, decode_times=False)
    ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180))

    mask_lon = (ds["lon"] >= min_lon) & (ds["lon"] <= max_lon)
    mask_lat = (ds["lat"] >= min_lat) & (ds["lat"] <= max_lat)
    subset = ds.where(mask_lon & mask_lat, drop=True).squeeze()

    u = subset["water_u"]
    v = subset["water_v"]
    ds.close()
    return xr.merge([u, v])

In [ ]:
from glob import glob


store = "/media/filipe/HDDJU/hycom"
fnames = sorted(glob(f"{store}/*.nc4"))

In [ ]:
datasets = [slice_daily_roi(fname) for fname in fnames]

In [ ]:
ds = xr.concat(datasets, dim="time")
ds.to_netcdf("hycom-oil.nc")

In [ ]:
[tmp.close() for tmp in datasets];
ds.close()